# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770845272487                   -0.53    9.0    156ms
  2   -2.772141691323       -2.89       -1.31    1.0    123ms
  3   -2.772170183601       -4.55       -2.58    1.0    107ms
  4   -2.772170689026       -6.30       -3.75    1.0    233ms
  5   -2.772170722229       -7.48       -4.09    2.0    166ms
  6   -2.772170722983       -9.12       -5.36    1.0    450ms
  7   -2.772170723013      -10.53       -5.39    2.0    139ms
  8   -2.772170723015      -11.67       -6.13    1.0    120ms
  9   -2.772170723015      -13.40       -6.47    2.0    141ms
 10   -2.772170723015      -13.92       -7.25    1.0    136ms
 11   -2.772170723015      -13.92       -8.07    1.0    134ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770659066573                   -0.53    9.0    202ms
  2   -2.7

1.7735578908022214

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770723236962                   -0.53    9.0    208ms
  2   -2.772054073399       -2.88       -1.31    1.0    124ms
  3   -2.772083012542       -4.54       -2.60    1.0    128ms
  4   -2.772083390503       -6.42       -3.96    1.0    120ms
  5   -2.772083417429       -7.57       -4.24    2.0    135ms
  6   -2.772083417793       -9.44       -5.31    1.0    116ms
  7   -2.772083417810      -10.78       -5.68    2.0    128ms
  8   -2.772083417811      -12.18       -6.52    1.0    123ms
  9   -2.772083417811      -13.77       -6.79    2.0    136ms
 10   -2.772083417811   +  -14.21       -7.55    1.0    159ms
 11   -2.772083417811      -13.97       -8.25    1.0    124ms

Polarizability via ForwardDiff:       1.7725349615914252
Polarizability via finite difference: 1.7735578908022214
